In [0]:
# Importing Libraries

In [0]:
import pandas as pd
import numpy as np

In [0]:
from zipfile import ZipFile
file_name="HumanActivityRecognition.zip"

with ZipFile(file_name,'r') as zip:
    zip.extractall()
    print("Done")

Done


In [0]:
# Activities are the class labels
# It is a 6 class classification
ACTIVITIES = {
    0: 'WALKING',
    1: 'WALKING_UPSTAIRS',
    2: 'WALKING_DOWNSTAIRS',
    3: 'SITTING',
    4: 'STANDING',
    5: 'LAYING',
}

# Utility function to print the confusion matrix
def confusion_matrix(Y_true, Y_pred):
    Y_true = pd.Series([ACTIVITIES[y] for y in np.argmax(Y_true, axis=1)])
    Y_pred = pd.Series([ACTIVITIES[y] for y in np.argmax(Y_pred, axis=1)])

    return pd.crosstab(Y_true, Y_pred, rownames=['True'], colnames=['Pred'])

### Data

In [0]:
# Data directory
DATADIR = 'UCI_HAR_Dataset'

In [0]:
# Raw data signals
# Signals are from Accelerometer and Gyroscope
# The signals are in x,y,z directions
# Sensor signals are filtered to have only body acceleration
# excluding the acceleration due to gravity
# Triaxial acceleration from the accelerometer is total acceleration
SIGNALS = [
    "body_acc_x",
    "body_acc_y",
    "body_acc_z",
    "body_gyro_x",
    "body_gyro_y",
    "body_gyro_z",
    "total_acc_x",
    "total_acc_y",
    "total_acc_z"
]

In [0]:
# Utility function to read the data from csv file
def _read_csv(filename):
    return pd.read_csv(filename, delim_whitespace=True, header=None)

# Utility function to load the load
def load_signals(subset):
    signals_data = []

    for signal in SIGNALS:
        filename = f'HAR/UCI_HAR_Dataset/{subset}/Inertial Signals/{signal}_{subset}.txt'
        signals_data.append(
            _read_csv(filename).to_numpy()
        ) 

    # Transpose is used to change the dimensionality of the output,
    # aggregating the signals by combination of sample/timestep.
    # Resultant shape is (7352 train/2947 test samples, 128 timesteps, 9 signals)
    return np.transpose(signals_data, (1, 2, 0))

In [0]:

def load_y(subset):
    """
    The objective that we are trying to predict is a integer, from 1 to 6,
    that represents a human activity. We return a binary representation of 
    every sample objective as a 6 bits vector using One Hot Encoding
    (https://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html)
    """
    filename = f'HAR/UCI_HAR_Dataset/{subset}/y_{subset}.txt'
    y = _read_csv(filename)[0]

    return pd.get_dummies(y).to_numpy()

In [0]:
def load_data():
    """
    Obtain the dataset from multiple files.
    Returns: X_train, X_test, y_train, y_test
    """
    X_train, X_test = load_signals('train'), load_signals('test')
    y_train, y_test = load_y('train'), load_y('test')

    return X_train, X_test, y_train, y_test

In [0]:
# Importing tensorflow
np.random.seed(42)
import tensorflow as tf
tf.compat.v1.set_random_seed(42)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

In [0]:
# Configuring a session
session_conf = tf.compat.v1.ConfigProto(
    intra_op_parallelism_threads=1,
    inter_op_parallelism_threads=1
)

In [0]:
# Import Keras
from keras import backend as K
sess = tf.compat.v1.Session(graph=tf.compat.v1.get_default_graph(), config=session_conf)
K.set_session(sess)
#tf.compat.v1.Session()

Using TensorFlow backend.


In [0]:
# Importing libraries
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers.core import Dense, Dropout

In [0]:
# Initializing parameters
epochs = 30
batch_size = 16
n_hidden = 32

In [0]:
# Utility function to count the number of classes
def _count_classes(y):
    return len(set([tuple(category) for category in y]))

In [0]:
# Loading the train and test data
X_train, X_test, Y_train, Y_test = load_data()

In [0]:
timesteps = len(X_train[0])
input_dim = len(X_train[0][0])
n_classes = _count_classes(Y_train)

print(timesteps)
print(input_dim)
print(len(X_train))

128
9
7352


In [0]:
!pip install tensorflow==1.14.0

     |████████████████████████████████| 109.2MB 77kB/s 
     |████████████████████████████████| 3.2MB 35.5MB/s 
     |████████████████████████████████| 491kB 50.9MB/s 
  Found existing installation: tensorboard 2.2.0
    Uninstalling tensorboard-2.2.0:
      Successfully uninstalled tensorboard-2.2.0
  Found existing installation: tensorflow-estimator 2.2.0rc0
    Uninstalling tensorflow-estimator-2.2.0rc0:
      Successfully uninstalled tensorflow-estimator-2.2.0rc0
  Found existing installation: tensorflow 2.2.0rc2
    Uninstalling tensorflow-2.2.0rc2:
      Successfully uninstalled tensorflow-2.2.0rc2


- Defining the Architecture of LSTM

In [0]:
# Initiliazing the sequential model
model = Sequential()
# Configuring the parameters
model.add(LSTM(n_hidden, input_shape=(timesteps, input_dim)))
# Adding a dropout layer
model.add(Dropout(0.5))
# Adding a dense output layer with sigmoid activation
model.add(Dense(n_classes, activation='sigmoid'))
model.summary()





Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 32)                5376      
_________________________________________________________________
dropout_1 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 6)                 198       
Total params: 5,574
Trainable params: 5,574
Non-trainable params: 0
_________________________________________________________________


In [0]:
# Compiling the model
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [0]:
# Training the model
model.fit(X_train,
          Y_train,
          batch_size=batch_size,
          validation_data=(X_test, Y_test),
          epochs=epochs)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Train on 7352 samples, validate on 2947 samples
Epoch 1/30
7352/7352 [==============================] - 30s 4ms/step - loss: 1.3207 - acc: 0.4329 - val_loss: 1.1474 - val_acc: 0.4706
Epoch 2/30
7352/7352 [==============================] - 29s 4ms/step - loss: 0.9877 - acc: 0.5702 - val_loss: 1.0286 - val_acc: 0.5046
Epoch 3/30
7352/7352 [==============================] - 31s 4ms/step - loss: 0.7947 - acc: 0.6477 - val_loss: 0.7681 - val_acc: 0.6074
Epoch 4/30
7352/7352 [==============================] - 30s 4ms/step - loss: 0.6952 - acc: 0.6578 - val_loss: 0.7221 - val_acc: 0.6060
Epoch 5/30
7352/7352 [==============================] - 30s 4ms/step - loss: 0.6491 - acc: 0.6802 - val_loss: 0.7290 - val_acc: 0.6169
Epoch 6/30
7352/7352 [==============================] - 31s 4ms/step - loss: 0.6374 - acc: 0.6857 - val_loss: 1.2811 - val_acc: 0.5877
Epoch 7/30
7352/7352 [==========================

In [0]:
# Confusion Matrix
print(confusion_matrix(Y_test, model.predict(X_test)))

Pred                LAYING  SITTING  ...  WALKING_DOWNSTAIRS  WALKING_UPSTAIRS
True                                 ...                                      
LAYING                 510        0  ...                   0                 2
SITTING                  1      417  ...                   0                 4
STANDING                 0      114  ...                   0                 2
WALKING                  0        0  ...                  21                32
WALKING_DOWNSTAIRS       1        2  ...                 400                15
WALKING_UPSTAIRS         0        5  ...                   4               448

[6 rows x 6 columns]


In [0]:
score = model.evaluate(X_test, Y_test)

2947/2947 [==============================] - 1s 317us/step


In [0]:
score

[0.4319933943883621, 0.8937902952154734]

- With a simple 2 layer architecture we got 89.37% accuracy and a loss of 0.43
- We can further imporve the performace with Hyperparameter tuning

**Tune the Dropout rate**

In [0]:
# Initiliazing the sequential model
model = Sequential()
# Configuring the parameters
model.add(LSTM(n_hidden, input_shape=(timesteps, input_dim)))
# Adding a dropout layer
model.add(Dropout(0.75))
# Adding a dense output layer with sigmoid activation
model.add(Dense(n_classes, activation='sigmoid'))
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 32)                5376      
_________________________________________________________________
dropout_1 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 6)                 198       
Total params: 5,574
Trainable params: 5,574
Non-trainable params: 0
_________________________________________________________________


In [0]:
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [0]:
model.fit(X_train,
          Y_train,
          batch_size=batch_size,
          validation_data=(X_test, Y_test),
          epochs=epochs)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Train on 7352 samples, validate on 2947 samples
Epoch 1/30
7352/7352 [==============================] - 16s 2ms/step - loss: 1.4256 - accuracy: 0.4010 - val_loss: 1.2302 - val_accuracy: 0.4530
Epoch 2/30
7352/7352 [==============================] - 16s 2ms/step - loss: 1.1568 - accuracy: 0.4918 - val_loss: 1.0141 - val_accuracy: 0.5402
Epoch 3/30
7352/7352 [==============================] - 16s 2ms/step - loss: 0.9729 - accuracy: 0.5789 - val_loss: 0.8321 - val_accuracy: 0.6138
Epoch 4/30
7352/7352 [==============================] - 17s 2ms/step - loss: 0.8530 - accuracy: 0.6163 - val_loss: 0.7815 - val_accuracy: 0.6043
Epoch 5/30
7352/7352 [==============================] - 17s 2ms/step - loss: 0.8117 - accuracy: 0.6359 - val_loss: 0.7919 - val_accuracy: 0.6101
Epoch 6/30
7352/7352 [==============================] - 16s 2ms/step - loss: 0.9681 - accuracy: 0.5851 - val_loss: 1.2472 - val_accu

In [0]:
# Confusion Matrix
print(confusion_matrix(Y_test, model.predict(X_test)))

Pred                LAYING  SITTING  ...  WALKING_DOWNSTAIRS  WALKING_UPSTAIRS
True                                 ...                                      
LAYING                 510        0  ...                   0                27
SITTING                  2      380  ...                   1                 1
STANDING                 0       77  ...                   0                 0
WALKING                  0        1  ...                  23                 9
WALKING_DOWNSTAIRS       0        0  ...                 298               115
WALKING_UPSTAIRS         0        0  ...                  24               417

[6 rows x 6 columns]


In [0]:
score = model.evaluate(X_test, Y_test)

2947/2947 [==============================] - 1s 315us/step


In [0]:
score

[0.5759321995358251, 0.8540889024734497]

In [0]:
model = Sequential()
# Configuring the parameters
model.add(LSTM(n_hidden, input_shape=(timesteps, input_dim)))
# Adding a dropout layer
model.add(Dropout(0.2))
# Adding a dense output layer with sigmoid activation
model.add(Dense(n_classes, activation='sigmoid'))
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_2 (LSTM)                (None, 32)                5376      
_________________________________________________________________
dropout_2 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 6)                 198       
Total params: 5,574
Trainable params: 5,574
Non-trainable params: 0
_________________________________________________________________


In [0]:
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [0]:
model.fit(X_train,
          Y_train,
          batch_size=batch_size,
          validation_data=(X_test, Y_test),
          epochs=epochs)

Train on 7352 samples, validate on 2947 samples
Epoch 1/30
7352/7352 [==============================] - 17s 2ms/step - loss: 1.2395 - accuracy: 0.4954 - val_loss: 0.9725 - val_accuracy: 0.5697
Epoch 2/30
7352/7352 [==============================] - 16s 2ms/step - loss: 0.8660 - accuracy: 0.6270 - val_loss: 1.2941 - val_accuracy: 0.4869
Epoch 3/30
7352/7352 [==============================] - 17s 2ms/step - loss: 0.7043 - accuracy: 0.7018 - val_loss: 0.7861 - val_accuracy: 0.6797
Epoch 4/30
7352/7352 [==============================] - 16s 2ms/step - loss: 0.5475 - accuracy: 0.7783 - val_loss: 0.6620 - val_accuracy: 0.7285
Epoch 5/30
7352/7352 [==============================] - 16s 2ms/step - loss: 0.4577 - accuracy: 0.8200 - val_loss: 0.6436 - val_accuracy: 0.8103
Epoch 6/30
7352/7352 [==============================] - 16s 2ms/step - loss: 0.3739 - accuracy: 0.8887 - val_loss: 0.5980 - val_accuracy: 0.7862
Epoch 7/30
7352/7352 [==============================] - 16s 2ms/step - loss: 0.272

In [0]:
# Confusion Matrix
print(confusion_matrix(Y_test, model.predict(X_test)))

Pred                LAYING  SITTING  ...  WALKING_DOWNSTAIRS  WALKING_UPSTAIRS
True                                 ...                                      
LAYING                 510        1  ...                   0                 0
SITTING                  0      407  ...                   0                 3
STANDING                 0      100  ...                   0                 0
WALKING                  0        0  ...                  25                 6
WALKING_DOWNSTAIRS       0        0  ...                 412                 6
WALKING_UPSTAIRS         0        0  ...                   2               446

[6 rows x 6 columns]


In [0]:
score = model.evaluate(X_test, Y_test)

2947/2947 [==============================] - 1s 315us/step


In [0]:
score

[0.43277596008456193, 0.9053274393081665]

1.Performing tuning on drop-out rate we get a good accuracy arround 90.53 than previous model 

2.We can simillary perform hyper-parameter tuning on different parameter